In [20]:
# Study: Natural Language Processing with Deep Learning
# Dataset: Dead By Daylight Steam Reviews
# Author: Willian Oliveira and Julierme Silva
# Start: 10/04/2023
# Study Motivation: Train a machine to classify products based on user reviews
# Notebook Motivation: The purpose of this notebook is to fine-tune a in-context language model to perform classification reviews for Dead By Daylight
# Study Status: Finished

In [21]:
# Importing the libraries

import pandas as pd
import time
import datetime
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import GPT2Tokenizer, GPT2Config, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.enabled = True


SEED = 0
set_seed(SEED)  # Setting seed for reproducibility

# Setting device to GPU if available, else CPU

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(
    f'Using Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU"}'
)

Using Device: NVIDIA GeForce RTX 3060


In [22]:
# Loading the dataset

df = pd.read_csv('data/raw/dbd_english_reviews.csv')
df = df.dropna()
df.head()

,review_id,review_text,recommended
0,136024101,good game\n,True
1,136022433,BHVR implemented dc penalty cos they apparentl...,False
2,136022116,เกมหมาๆ คิลเก่งก็เก่งไปเลย กากก็กากสัส กดโซโล่...,False
3,136019421,This game is very fun unless you get fucked by...,True
4,136019209,Why?,False


In [23]:
# Splitting the dataset into the Training, Validation and Test set with imbalanced-learn, splitting first and then undersampling to avoid data leakage

X = df["review_text"]
y = df["recommended"]

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=SEED
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=SEED
)

# Undersampling the training set

rus = RandomUnderSampler(random_state=SEED)
X_train, y_train = rus.fit_resample(X_train.values.reshape(-1, 1), y_train)

# Converting X_train resampled back to a string to be used by the tokenizer

X_train = np.ravel(X_train)

In [24]:
# Creating the context dataset

def create_context_dataset(reviews, labels, prepend_text="Review:"):
    context_data = []
    for review, label in zip(reviews, labels):
        context_data.append(f"{prepend_text} {review} {'Recommended' if label else 'Not recommended'}")
    return context_data

X_train_context = create_context_dataset(X_train, y_train)
X_val_context = create_context_dataset(X_val, y_val)
X_test_context = create_context_dataset(X_test, y_test)

In [25]:
# Load the GPT tokenizer.
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>') #gpt2-medium


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [26]:
# Verify tokenizer metrics

print("The max model length is {} for this model, although the actual embedding size for GPT small is 768".format(tokenizer.model_max_length))
print("The beginning of sequence token {} token has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.bos_token_id), tokenizer.bos_token_id))
print("The end of sequence token {} has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.eos_token_id), tokenizer.eos_token_id))
print("The padding token {} has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.pad_token_id), tokenizer.pad_token_id))

The max model length is 1024 for this model, although the actual embedding size for GPT small is 768
The beginning of sequence token <|startoftext|> token has the id 50257
The end of sequence token <|endoftext|> has the id 50256
The padding token <|pad|> has the id 50258


In [27]:
batch_size = 2

In [28]:
# Creating the dataset for the neural network

class GPT2Dataset(Dataset):

  def __init__(self, txt_list, tokenizer, gpt2_type="gpt2", max_length=768):

    self.tokenizer = tokenizer
    self.input_ids = []
    self.attn_masks = []

    for txt in txt_list:

      encodings_dict = tokenizer('<|startoftext|>'+ txt + '<|endoftext|>', truncation=True, max_length=max_length, padding="max_length")

      self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
      self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
    
  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.attn_masks[idx] 

In [18]:
train_dataset = GPT2Dataset(X_train_context, tokenizer, max_length=768)
val_dataset = GPT2Dataset(X_val_context, tokenizer, max_length=768)
test_dataset = GPT2Dataset(X_test_context, tokenizer, max_length=768)

print('{:>5,} training samples'.format(len(train_dataset)))
print('{:>5,} validation samples'.format(len(val_dataset)))
print('{:>5,} test samples'.format(len(test_dataset)))

46,178 training samples
25,985 validation samples
25,986 test samples


In [19]:
# Create the DataLoaders for our training, validation and test datasets.

train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )


validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

test_dataloader = DataLoader(
            test_dataset, # The test samples.
            sampler = SequentialSampler(test_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )




In [29]:
configuration = GPT2Config.from_pretrained('gpt2', output_hidden_states=False)

# instantiate the model
model = GPT2LMHeadModel.from_pretrained("gpt2", config=configuration)

# resize the token embeddings to match the new vocabulary size

model.resize_token_embeddings(len(tokenizer))

# move the model to the GPU

device = torch.device("cuda")
model.cuda()

epochs = 1
learning_rate = 5e-4
warmup_steps = 1e2
epsilon = 1e-8

# this produces sample output every 100 steps
sample_every = 100

optimizer = AdamW(model.parameters(),
                  lr = learning_rate,
                  eps = epsilon
                )

c:\Users\willi\anaconda3\envs\torch_1.13\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [30]:
# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = warmup_steps, 
                                            num_training_steps = total_steps)

In [29]:
def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

In [30]:
total_t0 = time.time()

training_stats = []

model = model.to(device)

for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    total_train_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        model.zero_grad()        

        outputs = model(  b_input_ids,
                          labels=b_labels, 
                          attention_mask = b_masks,
                          token_type_ids=None
                        )

        loss = outputs[0]  

        batch_loss = loss.item()
        total_train_loss += batch_loss

        # Get sample every x batches.
        if step % sample_every == 0 and not step == 0:

            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}. Loss: {:>5,}.   Elapsed: {:}.'.format(step, len(train_dataloader), batch_loss, elapsed))

            model.eval()

            sample_outputs = model.generate(
                                    bos_token_id=random.randint(1,30000),
                                    do_sample=True,   
                                    top_k=50, 
                                    max_length = 200,
                                    top_p=0.95, 
                                    num_return_sequences=1
                                )
            for i, sample_output in enumerate(sample_outputs):
                  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))
            
            model.train()

        loss.backward()

        optimizer.step()

        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)       
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)
        
        with torch.no_grad():        

            outputs  = model(b_input_ids, 
#                            token_type_ids=None, 
                             attention_mask = b_masks,
                            labels=b_labels)
          
            loss = outputs[0]  
            
        batch_loss = loss.item()
        total_eval_loss += batch_loss        

    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    validation_time = format_time(time.time() - t0)    

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 1 ========
Training...


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch   100  of  23,089. Loss: 1.4421923160552979.   Elapsed: 0:00:39.
0:  invitesReview: this game is a remake of A Tale of the Dying Dark
. If you've seen a scary or amazing game. It's an entertaining and fun game. If you like, you'll be playing this game. I want to make this game a game that will not suffer it's side side-scrolling hook and hook and hook game. If you enjoy this game, you should play this game. I want it to be. It is a game about a bad game and a horrible game and a lot of randomness has been made this year. If you enjoy this game, you should play the game. I hate making fun of the poor and the poor. It is not recommended or recommended for players and if you're any it's only for the playability of people. It's easy to get bored of playing and if you're ever upset there, I am here to talk about it. You can learn a lot by looking at these games, but I am trying to


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch   200  of  23,089. Loss: 1.4639204740524292.   Elapsed: 0:01:19.
0:  CCReview: In game: Not recommended


...

  Batch 22,800  of  23,089. Loss: 0.10989409685134888.   Elapsed: 2:25:42.
0: ensityReview: i hate this game Not recommended


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch 22,900  of  23,089. Loss: 0.010547954589128494.   Elapsed: 2:26:22.
0:  EnterReview: I'm not sure what to expect from this game, but I like that the games you play are good, especially with friends.

So the way the community turns this community out to be is very toxic.

Do you have a better idea of a game that I play?
Do you really like the gameplay?
Do you love horror games where you are literally trying to survive against an unstoppable killer?
Do you like asymmetrical multiplayer games where you play survivor?
Do you like getting chased around by the killer?
Do you like finding yourself stuck in a circle of grass?
Do you like hunting people?
Do you like that a player can be your enemy?
Do you like a game that forces you to play against them in the manner of stealth/stunned perks?
Do you like that being chased and feeling your power over the killer?
Do you like running around for your life?
Do you like that feeling?



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch 23,000  of  23,089. Loss: 0.4988093674182892.   Elapsed: 2:27:04.
0: urenReview: Love Recommended

  Average training loss: 0.37
  Training epoch took: 2:27:38

Running Validation...
  Validation Loss: 0.21
  Validation took: 0:24:18

Training complete!
Total training took 2:51:56 (h:mm:ss)


In [32]:
# Save the model
torch.save(model.state_dict(), 'models/gpt2_1epoch.pt')

In [8]:
# Load the model

configuration = GPT2Config.from_pretrained('gpt2', output_hidden_states=False)
model = GPT2LMHeadModel.from_pretrained("gpt2", config=configuration)
model.resize_token_embeddings(len(tokenizer))
model.load_state_dict(torch.load('models/gpt2_1epoch.pt'))

# Set model to gpu

model = model.to(device)

In [10]:
# Generate text from the tuned model

model.eval()

prompt = "<|startoftext|>"

generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
generated = generated.to(device)

print(generated)

sample_outputs = model.generate(
                                generated, 
                                bos_token_id=random.randint(1,30000),
                                do_sample=True,   
                                top_k=50, 
                                max_length = 300,
                                top_p=0.95, 
                                num_return_sequences=3
                                )

for i, sample_output in enumerate(sample_outputs):
  print("{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[50257]], device='cuda:0')
0: Review: Great game, especially the killer. But they can make it super difficult at times. Recommended


1: Review: Pretty good job so far Recommended


2: Review: This game is so bad for survivors that I can't join game. There's so many bugs and glitches. The game doesn't work at all.
If you're looking for an even remotely enjoyable experience, I really think the game is for you. Not recommended


